In [1]:
!pipenv install pandas numpy matplolib seaborn plotly scipy scikit-learn

Courtesy Notice:
Pipenv found itself running within a virtual environment,  so it will 
automatically use that environment, instead of  creating its own for any 
project. You can set
PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and 
create  its own instead.
You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing pandas...
✔ Installation Succeeded
Installing numpy...
✔ Installation Succeeded
Installing matplolib...
✔ Installation Succeeded
Installing seaborn...
✔ Installation Succeeded
Installing plotly...
✔ Installation Succeeded
Installing scipy...
✔ Installation Succeeded
Installing scikit-learn...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (22ffb7)...
All dependencies are now